# Example of the ABCD method with arbitrary dataset distributions.

This code block are the initialization and imports that are needed. Simulation parameters can be changed here as desired. Four correlations are coded between x and y.

0 is uncorrelated, and causes x and y to be completely unrelated.

1 is a simple linear function, y = x

2 is a simple quadratic function, y = x^2 with a coefficient to control the height of the function based on the x-mean.

3 is a simple exponential function, y = e^x with a coefficient to control the height of the function based on the x-mean.

The chi squared parameter controls the strength of the correlation. At 1, the correlation is perfect. For weak correlations, this should be above 1,000. Don't set this to less than 1.

In [64]:
import numpy as np
import random
import matplotlib.pyplot as plot
import math

random.seed()

# hardcoded values that are adjustable easily from here

xmean = 50 # average x coordinate in distribution
xstdv = 15 # deviation x coordinate in distro
ymean = 50 # average y coordinate in distro for uncorrelated
ystdv = 15 # deviation y coordinate in distro for uncorrelated

ycut = 60 # horizontal quadrant cut
xcut = 65 # vertical quadrant cut

events = 10000 # number of events to generate

fit_spread = 250 #controls how much spread the correlation function has. Higher = weaker correlation.

correlationfunction = 0 #choose which correlation function you want to use.

#initialize variables we'll need later

acount = 0 # events in signal sector
bcount = 0 # events in background 1
ccount = 0 # events in background 2
dcount = 0 # events in background 3

The following code block defines the various correlation functions that we can use, to be called later depending on if they're needed.

In [52]:
def uncorrelated(x, y):
    global acount
    global bcount
    global ccount
    global dcount
    x.append(random.gauss(xmean, xstdv))
    y.append(random.gauss(ymean, ystdv))   
    while x[i]<=0:
        x[i] = random.gauss(xmean, xstdv)
    #while y[i]<=0:
    #    y[i] = random.gauss(ymean, ystdv)
    if x[i]>=ycut and y[i]>=xcut:
        acount+=1
    if x[i]>=ycut and y[i]<=xcut:
        bcount+=1
    if x[i]<=ycut and y[i]<=xcut:
        ccount+=1
    if x[i]<=ycut and y[i]>=xcut:
        dcount+=1

def linear(x, y):
    global acount
    global bcount
    global ccount
    global dcount
    global rchis
    x.append(random.gauss(xmean, xstdv))
    while x[i]<=0:
        x[i] = random.gauss(xmean, xstdv)
    expy = x[i]+50
    y.append(random.gauss(expy, fit_spread*7/3))
    #while y[i]<=0:
    #    y[i] = random.gauss(expy, fit_spread*7/3)
    if x[i]>=ycut and y[i]>=xcut:
        acount+=1
    if x[i]>=ycut and y[i]<=xcut:
        bcount+=1
    if x[i]<=ycut and y[i]<=xcut:
        ccount+=1
    if x[i]<=ycut and y[i]>=xcut:
        dcount+=1
    dev = (y[i] - expy)**2/expy
    rchis += dev

def quadratic(x, y):
    global acount
    global bcount
    global ccount
    global dcount
    global rchis
    x.append(random.gauss(xmean, xstdv))
    while x[i]<=0:
        x[i] = random.gauss(xmean, xstdv)
    expy = x[i]**2.0/(xmean/2.0)
    y.append(random.gauss(expy, fit_spread*7/3))
    #while y[i]<=0:
    #    y[i] = random.gauss(expy, fit_spread*7/3)
    if x[i]>=ycut and y[i]>=xcut:
        acount+=1
    if x[i]>=ycut and y[i]<=xcut:
        bcount+=1
    if x[i]<=ycut and y[i]<=xcut:
        ccount+=1
    if x[i]<=ycut and y[i]>=xcut:
        dcount+=1
    dev = (y[i] - expy)**2/expy
    rchis += dev
                          
def exponential(x, y):
    global acount
    global bcount
    global ccount
    global dcount
    x.append(random.gauss(xmean, xstdv))
    while x[i]<=0:
        x[i] = random.gauss(xmean, xstdv)
    expy = math.exp(1+(7/3*x[i]/xmean))
    y.append(random.gauss(expy, fit_spread*7/3))
    #while y[i]<=0:
    #    y[i] = random.gauss(expy, fit_spread*7/3)
    if x[i]>=ycut and y[i]>=xcut:
        acount+=1
    if x[i]>=ycut and y[i]<=xcut:
        bcount+=1
    if x[i]<=ycut and y[i]<=xcut:
        ccount+=1
    if x[i]<=ycut and y[i]>=xcut:
        dcount+=1
        

Execute the following code block to actually generate data, count events in each quadrant, and perform the ABCD analysis. Notice that this code block resets itself each time, so it can be run repeatedly.

In [ ]:
# reset our coordinates for our random distributions
xcoordinates = []
ycoordinates = []

# reset the counts for the number of events per sector
acount = 0 # events in signal sector
bcount = 0 # events in background 1
ccount = 0 # events in background 2
dcount = 0 # events in background 3
rchis = 0

# sectors are counted clockwise from upper right:

# D          A

#       +

# C          B

# initialize the counting number
i = 0

# here we generate our coordinates for our random distributions, ensuring that all coordinates are positive. Not strictly necessary, but why not?
# we also count the number of events in each sector here, since we're already going over every event's coordinate.
for i in range(0, events):
    if correlationfunction == 0:
        uncorrelated(xcoordinates, ycoordinates)
    elif correlationfunction == 1:
        linear(xcoordinates, ycoordinates)
    elif correlationfunction == 2:
        quadratic(xcoordinates, ycoordinates)
    elif correlationfunction == 3:
        exponential(xcoordinates, ycoordinates)
    else:
        print("Error: no correlation function specified. Defaulting to uncorrelated.")
        uncorrelated(xcoordinates, ycoordinates)

# create the actual plots
Dataset = plot.scatter(xcoordinates, ycoordinates, marker='o', s=4)
plot.axvline(x=ycut, c='darkred', lw=1)
plot.axhline(y=xcut, c='darkred', lw=1)
plot.title("Arbitrary Dataset")
plot.xlabel("Weakly Correlated Variable 1")
plot.ylabel("Weakly Correlated variable 2")
plot.annotate("Region A",
             xy=(xcut,ycut),
              textcoords="offset points",
              xytext=(50,50),
              c='darkred'
             )
plot.annotate("Region B",
             xy=(xcut,ycut),
              textcoords="offset points",
              xytext=(50,-50),
              c='darkred'
             )
plot.annotate("Region C",
             xy=(xcut,ycut),
              textcoords="offset points",
              xytext=(-100,-50),
              c='darkred'
             )
plot.annotate("Region D",
             xy=(xcut,ycut),
              textcoords="offset points",
              xytext=(-100,50),
              c='darkred'
             )
plot.show()
# print the number of counts in each sector

print(" Events in sector A: " , acount, "\n", "Events in sector B: ", bcount, "\n", "Events in Sector C: ", ccount, "\n", "Events in sector D: ", dcount, "\n")
        

# calculate estimated background in sector A using ABCD method, and the Poisson error

estimatedacount = round(bcount * dcount / ccount, 2)
estimatederror = round(math.sqrt(estimatedacount), 2)

# print what ABCD tells us is the background in sector A.

print(" Estimated events in sector A using ABCD method: ", estimatedacount, u"\u00b1", estimatederror, "\n")
print(" Reduced chi squared value of distribution: ", rchis/events)